In [2]:
import numpy as np
import pandas as pd
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

MIN_MATCH_COUNT = 19
import cv2
import time
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


In [6]:
#завантажити нейромережу
model = load_model('ReNet')
IMG_SIZE = 128

In [7]:
#завантажити вхідні дані
X = np.zeros((363, IMG_SIZE, IMG_SIZE, 3))
Y = np.zeros((363, 4))
for i in range(121):
    img = cv2.imread(f"img/{i}.jpg")
    x = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    X[i] = x
    if(i <= 102):
        Y[i][0] = 1
    else:
        Y[i][3] = 1
for i in range(121):
    img = cv2.imread(f"img2/{i}.jpg")
    x = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    X[i+121] = x
    if(i <= 100):
        Y[i+121][1] = 1
    else:
        Y[i+121][3] = 1
for i in range(121):
    img = cv2.imread(f"img3/{i}.jpg")
    x = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    X[i+242] = x
    if(i <= 94):
        Y[i+242][2] = 1
    else:
        Y[i+242][3] = 1        

        
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0, test_size=0.2)

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))


number of training examples = 290
number of test examples = 73
X_train shape: (290, 128, 128, 3)
Y_train shape: (290, 4)
X_test shape: (73, 128, 128, 3)
Y_test shape: (73, 4)


In [8]:
#пройти нейромережею по вхідних даних та зібрати метрики
t1 = 0
t2 = 0
t3 = 0
t4 = 0
a1 = 0
a2 = 0
a3 = 0
a4 = 0
times = 0
for i in range(X_test.shape[0]):
    start_time = time.time()
    p = model.predict(np.expand_dims(X_test[i], axis=0))
    times += (time.time() - start_time)
    #for j in range(4):
        #if p[j] == np.max(p):
    yp = np.argmax(p)
    yt = np.argmax(Y_test[i])
    #print(f"{i+1}: {yp} {yt}")
    if yp == yt == 0: t1 += 1
    elif yp == yt == 1: t2 += 1
    elif yp == yt == 2: t3 += 1
    elif yp == yt == 3: t4 += 1
        
    if yp == 0: a1 += 1
    elif yp == 1: a2 += 1
    elif yp == 2: a3 += 1
    elif yp == 3: a4 += 1

sY = np.sum(Y_test, axis = 0)
print(f"detected car:\t\t{t1/sY[0]}")
print(f"detected stand:\t\t{t2/sY[1]}")
print(f"detected dosimeter:\t{t3/sY[2]}")
print(f"nothing detected:\t{t4/sY[3]}\n")

print(f"false positive car:\t{(a1 - t1)/a1}")
print(f"false positive stand:\t{(a2 - t2)/a2}")
print(f"false positive dosimeter:{(a3 - t3)/a3}")
print(f"false negative:\t\t{(a4 - t4)/a4}\n")

print(f"mean time:\t\t{times/X_test.shape[0]}s")

detected car:		0.8333333333333334
detected stand:		0.8
detected dosimeter:	0.5882352941176471
nothing detected:	1.0

false positive car:	0.047619047619047616
false positive stand:	0.0
false positive dosimeter:0.09090909090909091
false negative:		0.52

mean time:		0.05946555202954436s


In [30]:
#пройти нейромережею та дескриптором по відео, та записати відео з результатами

cap = cv2.VideoCapture("carvideo.mp4")
img1 = cv2.imread("img/0.jpg")

surf = cv2.xfeatures2d.SURF_create()
kp1, des1 = surf.detectAndCompute(img1, None)
#out = cv2.VideoWriter('classificator1.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 1, (1080, 1920))
out = cv2.VideoWriter('classificator.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20, (960, 540))
while(1):    
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
        x = cv2.resize(frame, (128, 128), interpolation = cv2.INTER_AREA)
        x = np.expand_dims(x, axis=0)
        x = x/255.0
        kp2, des2 = surf.detectAndCompute(frame, None)
        bf = cv2.BFMatcher()

        good = []
        if isinstance(des2, np.ndarray):
            matches = bf.knnMatch(des1, des2, k=2)
            for m,n in matches:
                if m.distance < 0.75*n.distance:
                    good.append(m)

        if (len(good)>=MIN_MATCH_COUNT):
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
            matchesMask = mask.ravel().tolist()

            h,w,c = img1.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,M)

            frame = cv2.polylines(frame,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        p = model.predict(x)
        obj = ("car","stand","dozimeter","none")
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, obj[np.argmax(p)], (10,50), font, 2, (0, 255, 0), 2, cv2.LINE_AA)
        out.write(frame)
        cv2.imshow("result", frame)
    else:
        break

    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

out.release()    
cap.release()
cv2.destroyAllWindows()

<VideoWriter 000001B99D952450>


In [19]:
shape

(540, 960, 3)

In [18]:
#збір метрик дескриптора SURF на основі даних 2 лабораторної
results_SURF_stand = pd.read_csv('csv/SURF_VladB_dataresult.csv', sep=',')
results_SURF_dosimeter = pd.read_csv('csv/SURF_VladK_dataresult.csv', sep=',')
results_SURF_car = pd.read_csv('csv/SURF_Egor_dataresult.csv', sep=',')

stand_positive = 100
stand_negative = 20

dosimeter_positive = 94
dosimeter_negative = 20

car_positive = 102
car_negative = 18

TP_SURF_dosimeter = results_SURF_dosimeter["MeanDist"].head(dosimeter_positive).count().sum()
TN_SURF_dosimeter = results_SURF_dosimeter["MeanDist"].tail(dosimeter_negative).isna().sum()
AN_SURF_dosimeter = results_SURF_dosimeter["MeanDist"].isna().sum()
AP_SURF_dosimeter = results_SURF_dosimeter["MeanDist"].count().sum()

TP_SURF_stand = results_SURF_stand["MeanDist"].head(stand_positive).count().sum()
TN_SURF_stand = results_SURF_stand["MeanDist"].tail(stand_negative).isna().sum()
AN_SURF_stand = results_SURF_stand["MeanDist"].isna().sum()
AP_SURF_stand = results_SURF_stand["MeanDist"].count().sum()

TP_SURF_car = results_SURF_car["MeanDist"].head(car_positive).count().sum()
TN_SURF_car = results_SURF_car["MeanDist"].tail(car_negative).isna().sum()
AN_SURF_car = results_SURF_car["MeanDist"].isna().sum()
AP_SURF_car = results_SURF_car["MeanDist"].count().sum()

TN = TN_SURF_dosimeter + TN_SURF_stand + TN_SURF_car
N = stand_negative + dosimeter_negative + car_negative

print("detected car:\t\t", round(TP_SURF_car/car_positive, 2))
print("detected stand:\t\t", round(TP_SURF_stand/stand_positive, 2))
print("detected dosimeter:\t", round(TP_SURF_dosimeter/dosimeter_positive, 2))
print("nothing detected:\t", round(TN/N, 2))

AN = AN_SURF_dosimeter + AN_SURF_stand + AN_SURF_car

print("\nfalse positive car:\t", round((AP_SURF_car-TP_SURF_car)/AP_SURF_car, 2))
print("false positive stand:\t", round((AP_SURF_stand-TP_SURF_stand)/AP_SURF_stand, 2))
print("false positive dosimeter:", round((AP_SURF_dosimeter-TP_SURF_dosimeter)/AP_SURF_dosimeter, 2))
print("false negative:\t\t", round((AN-TN)/AN, 2))

t = 0
t += results_SURF_car["Time"].dropna().mean()
t += results_SURF_stand["Time"].dropna().mean()
t += results_SURF_dosimeter["Time"].dropna().mean()
print("\nmean time:\t\t", t/3)

#print(f"mean time:\t\t{times/X.shape[0]}s")

detected car:		 0.85
detected stand:		 0.7
detected dosimeter:	 0.61
nothing detected:	 0.74

false positive car:	 0.0
false positive stand:	 0.09
false positive dosimeter: 0.12
false negative:		 0.66

mean time:		 0.6526016520467836


In [21]:
#збір метрик дескриптора ORB на основі даних 2 лабораторної
results_ORB_stand = pd.read_csv('https://raw.githubusercontent.com/Viltexon/CVPR-Lab/master/Lab%202/csv/results_ORB_stand.csv', sep=',')
results_ORB_dosimeter = pd.read_csv('https://raw.githubusercontent.com/Viltexon/CVPR-Lab/master/Lab%202/csv/results_ORB_calculator.csv', sep=',')
results_ORB_car = pd.read_csv('https://raw.githubusercontent.com/Viltexon/CVPR-Lab/master/Lab%202/csv/results_ORB_CarModel.csv', sep=',')

stand_positive = 100
stand_negative = 20

dosimeter_positive = 94
dosimeter_negative = 20

car_positive = 102
car_negative = 18

TP_ORB_dosimeter = results_SURF_dosimeter["MeanDist"].head(dosimeter_positive).count().sum()
TN_ORB_dosimeter = results_SURF_dosimeter["MeanDist"].tail(dosimeter_negative).isna().sum()
AN_ORB_dosimeter = results_SURF_dosimeter["MeanDist"].isna().sum()
AP_ORB_dosimeter = results_SURF_dosimeter["MeanDist"].count().sum()

TP_ORB_stand = results_ORB_stand["MeanDist"].head(stand_positive).count().sum()
TN_ORB_stand = results_ORB_stand["MeanDist"].tail(stand_negative).isna().sum()
AN_ORB_stand = results_ORB_stand["MeanDist"].isna().sum()
AP_ORB_stand = results_ORB_stand["MeanDist"].count().sum()

TP_ORB_car = results_ORB_car["MeanDist"].head(car_positive).count().sum()
TN_ORB_car = results_ORB_car["MeanDist"].tail(car_negative).isna().sum()
AN_ORB_car = results_ORB_car["MeanDist"].isna().sum()
AP_ORB_car = results_ORB_car["MeanDist"].count().sum()

TN = TN_ORB_dosimeter + TN_ORB_stand + TN_ORB_car
N = stand_negative + dosimeter_negative + car_negative

print("detected car:\t\t", round(TP_ORB_car/car_positive, 2))
print("detected stand:\t\t", round(TP_ORB_stand/stand_positive, 2))
print("detected dosimeter:\t", round(TP_ORB_dosimeter/dosimeter_positive, 2))
print("nothing detected:\t", round(TN/N, 2))

AN = AN_ORB_dosimeter + AN_ORB_stand + AN_ORB_car

print("\nfalse positive car:\t", round((AP_ORB_car-TP_ORB_car)/AP_ORB_car, 2))
print("false positive stand:\t", round((AP_ORB_stand-TP_ORB_stand)/AP_ORB_stand, 2))
print("false positive dosimeter:", round((AP_ORB_dosimeter-TP_ORB_dosimeter)/AP_ORB_dosimeter, 2))
print("false negative:\t\t", round((AN-TN)/AN, 2))

t = 0
t += results_ORB_car["Time"].dropna().mean()
t += results_ORB_stand["Time"].dropna().mean()
t += results_ORB_dosimeter["Time"].dropna().mean()
print("\nmean time:\t\t", t/3)

#print(f"mean time:\t\t{times/X.shape[0]}s")

detected car:		 0.76
detected stand:		 0.72
detected dosimeter:	 0.61
nothing detected:	 0.76

false positive car:	 0.02
false positive stand:	 0.08
false positive dosimeter: 0.1
false negative:		 0.67

mean time:		 0.11481225399943124


In [22]:
#збір метрик дескриптора AKAZE на основі даних 2 лабораторної
results_AKAZE_stand = pd.read_csv('https://raw.githubusercontent.com/Viltexon/CVPR-Lab/master/Lab%202/csv/results_AKAZE_stand.csv', sep=',')
results_AKAZE_dosimeter = pd.read_csv('https://raw.githubusercontent.com/Viltexon/CVPR-Lab/master/Lab%202/csv/results_AKAZE_calculator.csv', sep=',')
results_AKAZE_car = pd.read_csv('https://raw.githubusercontent.com/Viltexon/CVPR-Lab/master/Lab%202/csv/results_AKAZE_CarModel.csv', sep=',')

stand_positive = 100
stand_negative = 20

dosimeter_positive = 94
dosimeter_negative = 20

car_positive = 102
car_negative = 18

TP_AKAZE_dosimeter = results_AKAZE_dosimeter["MeanDist"].head(dosimeter_positive).count().sum()
TN_AKAZE_dosimeter = results_AKAZE_dosimeter["MeanDist"].tail(dosimeter_negative).isna().sum()
AN_AKAZE_dosimeter = results_AKAZE_dosimeter["MeanDist"].isna().sum()
AP_AKAZE_dosimeter = results_AKAZE_dosimeter["MeanDist"].count().sum()

TP_AKAZE_stand = results_AKAZE_stand["MeanDist"].head(stand_positive).count().sum()
TN_AKAZE_stand = results_AKAZE_stand["MeanDist"].tail(stand_negative).isna().sum()
AN_AKAZE_stand = results_AKAZE_stand["MeanDist"].isna().sum()
AP_AKAZE_stand = results_AKAZE_stand["MeanDist"].count().sum()

TP_AKAZE_car = results_AKAZE_car["MeanDist"].head(car_positive).count().sum()
TN_AKAZE_car = results_AKAZE_car["MeanDist"].tail(car_negative).isna().sum()
AN_AKAZE_car = results_AKAZE_car["MeanDist"].isna().sum()
AP_AKAZE_car = results_AKAZE_car["MeanDist"].count().sum()

TN = TN_AKAZE_dosimeter + TN_AKAZE_stand + TN_AKAZE_car
N = stand_negative + dosimeter_negative + car_negative

print("detected car:\t\t", round(TP_AKAZE_car/car_positive, 2))
print("detected stand:\t\t", round(TP_AKAZE_stand/stand_positive, 2))
print("detected dosimeter:\t", round(TP_AKAZE_dosimeter/dosimeter_positive, 2))
print("nothing detected:\t", round(TN/N, 2))

AN = AN_AKAZE_dosimeter + AN_AKAZE_stand + AN_AKAZE_car

print("\nfalse positive car:\t", round((AP_AKAZE_car-TP_AKAZE_car)/AP_AKAZE_car, 2))
print("false positive stand:\t", round((AP_AKAZE_stand-TP_AKAZE_stand)/AP_AKAZE_stand, 2))
print("false positive dosimeter:", round((AP_AKAZE_dosimeter-TP_AKAZE_dosimeter)/AP_AKAZE_dosimeter, 2))
print("false negative:\t\t", round((AN-TN)/AN, 2))

t = 0
t += results_AKAZE_car["Time"].dropna().mean()
t += results_AKAZE_stand["Time"].dropna().mean()
t += results_AKAZE_dosimeter["Time"].dropna().mean()
print("\nmean time:\t\t", t/3)

#print(f"mean time:\t\t{times/X.shape[0]}s")

detected car:		 0.72
detected stand:		 0.71
detected dosimeter:	 0.61
nothing detected:	 0.71

false positive car:	 0.05
false positive stand:	 0.09
false positive dosimeter: 0.1
false negative:		 0.7

mean time:		 0.33894726254089536
